# Patient Data Query

This notebook queries patient data from multiple databases for a given prontuario.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_embrioes_congelados
- **Finops Silver Layer** (huntington_data_lake.duckdb):
  - silver.mesclada_vendas


In [1]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Input: Patient Prontuario


In [2]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Database Connections


In [3]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
        # if not df.empty:
        #     display(df)
        # else:
        #     print("No records found.")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [4]:
def query_biopsy_pgta_timeline(conn, prontuario):
    """Query biopsy and PGT-A timeline data for a specific prontuario"""
    print(f"\n=== BIOPSY PGT-A TIMELINE ===")
    query = f"""
    SELECT * FROM gold.biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_billing_timeline(conn, prontuario):
    """Query billing timeline data for a specific prontuario"""
    print(f"\n=== BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in billing_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [5]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_semen(conn, prontuario):
    """Query view_congelamentos_semen for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_SEMEN ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_semen 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_semen")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_mesclada_vendas(conn, prontuario):
    """Query silver.mesclada_vendas for a specific prontuario"""
    print(f"\n=== SILVER.MESCLADA_VENDAS ===")
    query = f"""
    SELECT * FROM silver.mesclada_vendas 
    WHERE prontuario = {prontuario}
    ORDER BY "DT Emissao" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.mesclada_vendas")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [6]:
def query_comprehensive_timeline(conn, prontuario):
    """Query comprehensive timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [7]:
def query_resumed_timeline(conn, prontuario):
    """Query resumed timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [8]:
def query_embryoscope_timeline(conn, prontuario):
    """Query embryoscope timeline data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_embryoscope_timeline(conn, prontuario):
    """Query resumed embryoscope timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED EMBRYOSCOPE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_embryoscope_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_treatment (conn, prontuario):
    """Query embryoscope treatment data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TREATMENT ===")
    query = f"""
    SELECT 
        patient_PatientIDx
        ,	patient_PatientID
        ,	patient_FirstName	
        , patient_LastName	
        , patient_unit_huntington	
        , embryo_KIDDate
        , embryo_FertilizationTime 
        , treatment_TreatmentName	
        , patient_unit_huntington 
    FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryoscope_embrioes")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_freeze_timeline(conn, prontuario):
    """Query embryo freeze timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_freeze_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_freeze_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_billing_timeline(conn, prontuario):
    """Query embryo billing timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_billing_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_comprehensive_embryo_timeline(conn, prontuario):
    """Query comprehensive embryo timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_embryo_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_embryo_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in cryopreservation_events_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.resumed_cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_cryopreservation_events_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_consultas_timeline(conn, prontuario):
    """Query consultation timeline data for a specific prontuario"""
    print(f"\n=== CONSULTAS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_consultas_timeline(conn, prontuario):
    """Query resumed consultation timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED CONSULTAS TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


def get_finops_data(conn, prontuario):
    """Get finops data for a specific patient"""
    print(f"\n=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===")

    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {prontuario}
    """

    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [9]:
# Updated query function with biopsy PGT-A timeline and cryopreservation data
def query_patient_data_with_timeline(prontuario):
    """
    Query all patient data including timeline data from gold layer tables.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results including timeline data
    """
    print(f"\n{'='*80}")
    print(f"QUERYING DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_embrioes = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_congelamentos_semen = query_congelamentos_semen(clinisys_con, prontuario)
        
        # Query mesclada table
        df_mesclada = query_mesclada_vendas(huntington_con, prontuario)

        # query Tratamentos timeline
        df_tratamentos_timeline = get_finops_data(huntington_con, prontuario)
        # df_billing = query_billing_timeline(huntington_con, prontuario)
        df_comprehensive = query_comprehensive_timeline(huntington_con, prontuario)
        df_resumed = query_resumed_timeline(huntington_con, prontuario)
        
        
        # Query biopsy PGT-A timeline
        df_biopsy_pgta = query_biopsy_pgta_timeline(huntington_con, prontuario)
        
        
        
        # Query embryoscope timeline
        df_embryoscope_timeline = query_embryoscope_timeline(huntington_con, prontuario)
        df_resumed_embryoscope = query_resumed_embryoscope_timeline(huntington_con, prontuario)
        
        # Query embryoscope usage
        df_embryoscope = query_embryo_treatment(huntington_con, prontuario)
        
        # Query cryopreservation timeline data
        df_embryo_freeze_timeline = query_embryo_freeze_timeline(huntington_con, prontuario)
        df_embryo_billing_timeline = query_embryo_billing_timeline(huntington_con, prontuario)
        df_comprehensive_embryo_timeline = query_comprehensive_embryo_timeline(huntington_con, prontuario)
        
        
        df_cryopreservation_events_timeline = query_cryopreservation_events_timeline(huntington_con, prontuario)
        df_resumed_cryopreservation_events_timeline = query_resumed_cryopreservation_events_timeline(huntington_con, prontuario)

        # Query consultation timeline data
        df_consultas_timeline = query_consultas_timeline(huntington_con, prontuario)
        df_resumed_consultas_timeline = query_resumed_consultas_timeline(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_congelamentos_semen: {len(df_congelamentos_semen)} records")
        print(f"- silver.mesclada_vendas: {len(df_mesclada)} records")
        print(f"- tratamentos_timeline: {len(df_tratamentos_timeline)} records")
        print(f"- biopsy_pgta_timeline: {len(df_biopsy_pgta)} records")
        # print(f"- billing_timeline: {len(df_billing)} records")
        print(f"- comprehensive_timeline: {len(df_comprehensive)} records")
        print(f"- resumed_timeline: {len(df_resumed)} records")
        print(f"- embryoscope_timeline: {len(df_embryoscope_timeline)} records")
        print(f"- resumed_embryoscope_timeline: {len(df_resumed_embryoscope)} records")
        print(f"- embryoscope_treatment: {len(df_embryoscope)} records")
        print(f"- embryo_freeze_timeline: {len(df_embryo_freeze_timeline)} records")
        print(f"- embryo_billing_timeline: {len(df_embryo_billing_timeline)} records")
        print(f"- comprehensive_embryo_timeline: {len(df_comprehensive_embryo_timeline)} records")
        print(f"- cryopreservation_events_timeline: {len(df_cryopreservation_events_timeline)} records")
        print(f"- resumed_cryopreservation_events_timeline: {len(df_resumed_cryopreservation_events_timeline)} records")
        print(f"- consultas_timeline: {len(df_consultas_timeline)} records")
        print(f"- resumed_consultas_timeline: {len(df_resumed_consultas_timeline)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'embrioes_congelados': df_embrioes,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'congelamentos_semen': df_congelamentos_semen,
            'mesclada_vendas': df_mesclada,
            'tratamentos_timeline': df_tratamentos_timeline,
            'biopsy_pgta_timeline': df_biopsy_pgta,
            # 'billing_timeline': df_billing,
            'comprehensive_timeline': df_comprehensive,
            'resumed_timeline': df_resumed,
            'embryoscope_timeline': df_embryoscope_timeline,
            'resumed_embryoscope_timeline': df_resumed_embryoscope,
            'embryoscope_treatment': df_embryoscope,
            'embryo_freeze_timeline': df_embryo_freeze_timeline,
            'embryo_billing_timeline': df_embryo_billing_timeline,
            'comprehensive_embryo_timeline': df_comprehensive_embryo_timeline,
            'cryopreservation_events_timeline': df_cryopreservation_events_timeline,
            'resumed_cryopreservation_events_timeline': df_resumed_cryopreservation_events_timeline,
            'consultas_timeline': df_consultas_timeline,
            'resumed_consultas_timeline': df_resumed_consultas_timeline
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Query Finops Silver Layer Table


In [10]:
## Example usage with comprehensive timeline data including cryopreservation


prontuario = '788592'
prontuario = 823589


# Query all patient data including comprehensive timeline and cryopreservation data
results_with_timeline = query_patient_data_with_timeline(prontuario)


QUERYING DATA FOR PRONTUARIO: 823589
Query executed at: 2025-11-21 16:12:25.788615

=== VIEW_MICROMANIPULACAO ===
Found 1 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 12 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 2 records in view_tratamentos

=== VIEW_EMBRIOES_CONGELADOS ===
Found 6 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 1 records in view_congelamentos_embrioes

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 0 records in view_congelamentos_ovulos

=== VIEW_CONGELAMENTOS_SEMEN ===
Found 0 records in view_congelamentos_semen

=== SILVER.MESCLADA_VENDAS ===
Found 40 records in silver.mesclada_vendas

=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===
Found 1 records in resumed_consultas_timeline

=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===
Found 1 records in comprehensive_biopsy_pgta_timeline

=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===
Found 1 records 

# 0. Pagamentos

In [11]:
# Billing data
print("=== BILLING DATA ===")
mesclada_data = results_with_timeline['mesclada_vendas']
print(f"Records: {len(mesclada_data)}")
if not mesclada_data.empty:
    display(mesclada_data)
else:
    print("No billing data found.")


=== BILLING DATA ===
Records: 40


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-10-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,38793,RPS,38643,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,17902,20251118_150534,Mescladas2 (7).xlsx
1,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-09-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,33945,RPS,33816,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,36882,20251118_150534,Mescladas2 (7).xlsx
2,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-08-26,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,28961,RPS,28849,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,438225,20251118_150534,Mescladas2 (7).xlsx
3,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-07-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,131.0,Criopreservação,10150,502,22742,RPS,22642,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,431142,20251118_150534,Mescladas2 (7).xlsx
4,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-06-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10150,502,17405,RPS,17321,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,425084,20251118_150534,Mescladas2 (7).xlsx
5,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-05-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10150,502,12479,RPS,12423,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,419611,20251118_150534,Mescladas2 (7).xlsx
6,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-04-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10150,502,7389,RPS,7371,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,413849,20251118_150534,Mescladas2 (7).xlsx
7,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-03-24,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10150,502,3013,RPS,2995,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,409045,20251118_150534,Mescladas2 (7).xlsx
8,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-02-24,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10101,502,242442,RPS,241001,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,403383,20251118_150534,Mescladas2 (7).xlsx
9,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2025-01-23,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10101,502,237436,RPS,235986,0,CRIOPRESERVACAO,874491,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,39772

# 1. Tratamentos

In [12]:
# Treatments timeline data
print("=== TREATMENTS TIMELINE DATA ===")
tratamentos_timeline_data = results_with_timeline['tratamentos_timeline']
print(f"Records: {len(tratamentos_timeline_data)}")
if not tratamentos_timeline_data.empty:
    display(tratamentos_timeline_data)
else:
    print("No treatments timeline data found.")


=== TREATMENTS TIMELINE DATA ===
Records: 1


,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,prontuario_genitores,medico_nome,timeline_unidade,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments
0,823589,1.0,1.0,2.0,1.0,24700.0,2024-02-18,2024-05-10,2024-01-15,2024-01-15,823589,Thais Sanches Domingues,1. HTT SP - Ibirapuera,0,0,1,0,0


In [13]:
# Treatments data
print("=== TREATMENTS DATA ===")
tratamentos_data = results_with_timeline['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No treatments data found.")


=== TREATMENTS DATA ===
Records: 2


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,tipo_procedimento,tipo_inducao,protocolo,tipo_preparo_endometrial,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,utero_substituicao,prontuario_genitores,doacao_ovulos,doacao_embrioes,pgt,era_auxiliar,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,foliculos_antrais,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,data_inicio_agonista,utilizado_hcg,data_hcg,horario_planejado_hcg,gonapeptyl,dose_gonapeptyl,data_inicio_progesterona,progesterona_vaginal,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,motivo_nao_transferir,resultado_tratamento,analise_genetica,altura_bebe1,via_parto_bebe1,altura_bebe2,via_parto_bebe2,altura_bebe3,via_parto_bebe3,altura_bebe4,via_parto_bebe4,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,26793,823589,1,38,40,esposa,2,2024-05-10,Ciclo de Congelados,None,None,Natural,None,None,NaN,None,None,None,Homólogo,Não,NaN,None,None,PGT-A,Não,Não,Não,None,None,None,None,NaN,1.6,None,2024-04-22,NaN,NaN,None,Fator masculino,Outros,None,2024-04-22,19,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,5,2024-05-10,None,None,None,Gestação Clínica,Não,None,None,None,None,None,None,None,None,None,3673,31,be2bbe8faa5f21d642725433e5ba3ac5,2025-07-21 21:38:38
1,25092,823589,1,38,39,esposa,1,2024-02-18,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,PGT-A,None,None,None,None,None,Sim,None,NaN,1.6,None,2024-02-03,NaN,NaN,None,Outros,Fator masculino,None,2024-02-05,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Criopreservação total,No transfer,Não,None,None,None,None,None,None,None,None,None,3692,31,813de96ef73dc37eaec6e22a138d19b9,2025-07-21 21:38:38


In [14]:
# Billing data
print("=== BILLING DATA TREATMENTS===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Coleta - Adicional', 
    'Coleta - Crio', 
    'FET / FOT', 
    'Pró-Fiv', 
    'FIV', 
    'Inseminação', 
    'Ovodoação', 
    'Embriodoação'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA TREATMENTS===
Records: 1


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
37,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2024-01-15,FIV - MEDICOS INTERNOS,1.0,24700.0,FIV,10101,502,182614,RPS,181411,23,THAIS SANCHES DOMINGUES,874491,,S3551,0.0,0.0,0.0,4130.0,Ibirapuera,0,0,31110100005,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,249062,20251118_150534,Mescladas2 (7).xlsx


# 2. Biópsia - PGT-A

In [15]:
# Comprehensive timeline data
print("=== COMPREHENSIVE TIMELINE DATA ===")
comprehensive_timeline_data = results_with_timeline['comprehensive_timeline']
print(f"Records: {len(comprehensive_timeline_data)}")
if not comprehensive_timeline_data.empty:
    display(comprehensive_timeline_data)
else:
    print("No comprehensive timeline data found.")


=== COMPREHENSIVE TIMELINE DATA ===
Records: 1


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,823589,2024-02,6,6.0,11280.0,1,6.0,6.0,0.0,6,6.0,6600.0,1,6.0,6.0,0.0


In [16]:
# Resumed timeline data
print("=== RESUMED TIMELINE DATA ===")
resumed_timeline_data = results_with_timeline['resumed_timeline']
print(f"Records: {len(resumed_timeline_data)}")
if not resumed_timeline_data.empty:
    display(resumed_timeline_data)
else:
    print("No resumed timeline data found.")


=== RESUMED TIMELINE DATA ===
Records: 1


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,823589,2024-02,6,6.0,11280.0,1,6.0,6.0,0.0,6,6.0,6600.0,1,6.0,6.0,0.0


In [17]:
# Billing data
print("=== BILLING DATA BIOPSY===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Biópsia Embrionária',
    'PGT-A'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA BIOPSY===
Records: 2


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
25,823589,JULIANA UETSUK,823589,JULIANA UETSUKI MARTARELLI,823589,2024-02-29,ANALISE DE EMBRIOES PGT-A,6.0,6600.0,PGT-A,10101,502,189213,RPS,187941,23,THAIS SANCHES DOMINGUES,874489,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100024,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,256527,20251118_150534,Mescladas2 (7).xlsx
26,823589,JULIANA UETSUK,823589,JULIANA UETSUKI MARTARELLI,823589,2024-02-29,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,6.0,11280.0,Biópsia Embrionária,10101,502,189213,RPS,187941,23,THAIS SANCHES DOMINGUES,874489,,S3563.09,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100017,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,256528,20251118_150534,Mescladas2 (7).xlsx


In [18]:
# Micromanipulation oocytes data
print("=== MICROMANIPULATION OOCYTES DATA ===")
oocitos_data = results_with_timeline['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulation oocytes data found.")


=== MICROMANIPULATION OOCYTES DATA ===
Records: 12


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,Embriologista,PI,TCD,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,OocitoDoado,OrigemOocito,InseminacaoOocito,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,NClivou_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,Compactando_D6,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,184366,19404,Não,MII,None,3505,None,Descartado,None,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,7,3,Simétricos,Sim,Blastocisto Grau 1,None,None,None,Blastocisto Grau 2,88a64fb9adc0d3ee1cfe6d5a1ec8ab96,2025-07-21 21:40:02,1,1
1,184367,19404,Não,MII,None,3505,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,7,2,Simétricos,Sim,Blastocisto Grau 5,B,B,None,None,307747a99b3ce48c0f88103900facd38,2025-07-21 21:40:02,1,2
2,184368,19404,Não,MII,None,3505,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,8,2,Simétricos,Sim,Blastocisto Grau 5,A,A,None,None,f00ec94e0184f821c1ae0ab0a81df525,2025-07-21 21:40:02,1,3
3,184369,19404,Não,MII,None,3505,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,Mórula A,None,None,Sim,Blastocisto Grau 4,A,B,None,None,2f8865a770f7d4158423d38debd70885,2025-07-21 21:40:02,1,4
4,184370,19404,Não,MII,None,3505,3548,Criopreservado,Sim,Normal,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,10,1,Simétricos,Sim,Blastocisto Grau 5,A,A,None,None,38743eadb99c8b299e17fd13307dd4a7,2025-07-21 21:40:02,1,5
5,184371,19404,Não,MII,None,3505,None,Descartado,None,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,4,3,Simétricos,Sim,Fragmentação 3,None,None,Não,None,a2b07d5946fbbabb1411bb49c8a2e723,2025-07-21 21:40:02,1,6
6,184372,19404,Não,MII,None,3505,None,Descartado,None,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,5,3,Simétricos,Não,None,None,None,None,None,bb778c154be5838477ecb1e106eeacc4,2025-07-21 21:40:02,1,7
7,184373,19404,Não,MII,None,3505,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,Mórula B,None,None,Sim,Blastocisto Grau 5,B,B,None,None,a614245834417811d0a0005a65f699fc,2025-07-21 21:40:02,1,8
8,184374,19404,Não,MII,None,3505,None,Descartado,None,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,Mfrag,None,None,Não,None,None,None,None,None,0e9173d0ebea59180e1f44e58a2b6cbb,2025-07-21 21:40:02,1,9
9,184375,19404,Não,MII,None,3505,5997,Criopreservado,Sim,Normal,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,6,2,Simétricos,Sim,Fragmentação 2,None,None,Sim,Blastocisto Grau 4,d3a859658624fca8a5e58c7780d0fddd,2025-07-21 21:40:02,1,10


In [19]:
# Frozen embryos data
print("=== FROZEN EMBRYOS DATA ===")
embrioes_data = results_with_timeline['embrioes_congelados']
print(f"Records: {len(embrioes_data)}")
if not embrioes_data.empty:
    display(embrioes_data)
else:
    print("No frozen embryos data found.")


=== FROZEN EMBRYOS DATA ===
Records: 6


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,embriao,qualidade,comentarios,comentarios_descongelamento,ah,dia5_2,dia5_3,dia5_4,transferidos,pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,69971,184375,24675,0,823589,P10,E20748,#10,Blastocisto Grau 4 - B - B,None,None,None,None,None,None,None,None,Normal/Euplóide Masculino,Sim,Normal,3.8,D6,7.8,None,None,None,56c698484bffc20dd376f8a98c6b9b19,2025-07-21 21:38:57
1,69970,184373,24675,0,823589,P8,E20748,#8,Blastocisto Grau 5 - B - B,None,None,None,None,None,None,Descartado,None,"Aneuplóide complexo Monossomia 19, Trissomia 16 Feminino",Sim,Alterado,8.1,D5,2.1,None,None,None,4f12b693fa7ded68a2c7060628defcfe,2025-07-21 21:38:57
2,69969,184370,24675,14763,823589,P5,E20747,#5,Blastocisto Grau 5 - A - A,None,"EUPLOIDE, Feminino",Sim,Blastocisto Grau 5,A,A,Transferido,Sim,Normal/Euplóide Feminino,Sim,Normal,9.7,D5,2.58,None,None,None,59d3eb2ab5662afda897880d4d7cc2ab,2025-07-21 21:38:57
3,69968,184369,24675,0,823589,P4,E20747,#4,Blastocisto Grau 4 - A - B,None,None,None,None,None,None,Descartado,None,Aneuplóide complexo Trissomia 15 e 16 Feminino,Sim,Alterado,7.9,D5,8.51,None,None,None,05592ee9c3a4e37c6580ede87d80c364,2025-07-21 21:38:57
4,69967,184368,24675,0,823589,P3,E20747,#3,Blastocisto Grau 5 - A - A,None,None,None,None,None,None,Descartado,None,Anormal/Aneuplóide Monossomia 22 Feminino,Sim,Alterado,8.9,D5,2.02,None,None,None,2e73901847fd382dacd1a8547a66834d,2025-07-21 21:38:57
5,69966,184367,24675,0,823589,P2,E20747,#2,Blastocisto Grau 5 - B - B,None,None,None,None,None,None,None,None,Anormal/Aneuplóide Monossomia parcial 1q21.2q44 (99Mb) Masculino,Sim,Alterado,7.4,D5,1.58,None,None,None,5c816882096ecf0fe17a63186322551f,2025-07-21 21:38:57


In [20]:
# Display each result in separate cells

# Micromanipulation data
print("=== MICROMANIPULATION DATA ===")
micromanipulacao_data = results_with_timeline['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulation data found.")


=== MICROMANIPULATION DATA ===
Records: 1


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,recepcao_ovulos,Denud_DL,checagem_fertilizacao,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,ICSIDescongelados,pgd_oocito,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,unidade_id,responsavel_denud_id,profissional_fertilizacao_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,materiais_insumos,local_coleta,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,observacoes_resumo_ciclos,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,conc_ps1,EstadoSptz,check_gradiente_ps,check_swimup,volume_ps2,concentracao_progressivos_pos,conc_ps2,comentarios_ps,responsavel_ps,timelapse,transferidos,Transfer_D5,medico_responsavel_id,azoospermia,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,19404,458/24,823589,38,39,2024-02-18,12:45,13:00,10:10,31,4318,Não,12:30,08:00,08:00,08:00,08:00,08:00,11,12,Não,1,1,2024-02-18,ICSI,10 espermatozoides móveis progressivos\r\n,10 espermatozoides normais\r\n\r\n,None,0000000001,3652,5997,3505,5997,4005,4005,None,Laboratório,Normal,"0,85",02,03,1,None,None,None,None,None,None,None,ES 1,03,None,None,Ejaculado,10:46,11:30,"2,4",Não,"2,6","42,0",Fresco,None,None,"0,2","3,0","3,0",None,3529,Sim,0,Sim,31,Nenhum,None,None,None,esposa,10,10,3,0,10,0,0,0,0,0,3,0,0,0,0,0,0,IB233/24,None,1cb78ba62ec30ea9ef7b1179e2ad50e2,2025-07-21 21:39:47


# 3. Embryoscope

In [21]:
# Embryoscope timeline data
print("=== EMBRYOSCOPE TIMELINE DATA ===")
embryoscope_timeline_data = results_with_timeline['embryoscope_timeline']
print(f"Records: {len(embryoscope_timeline_data)}")
if not embryoscope_timeline_data.empty:
    display(embryoscope_timeline_data)
else:
    print("No embryoscope timeline data found.")


=== EMBRYOSCOPE TIMELINE DATA ===
Records: 1


,prontuario,period_month,embryoscope_usage_count,cumulative_embryoscope_usage,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,embryoscope_missing_payment
0,823589,2024-02,1,1.0,1,2500.0,1.0,2500.0,0.0


In [22]:
# Resumed embryoscope timeline data
print("=== RESUMED EMBRYOSCOPE TIMELINE DATA ===")
resumed_embryoscope_data = results_with_timeline['resumed_embryoscope_timeline']
print(f"Records: {len(resumed_embryoscope_data)}")
if not resumed_embryoscope_data.empty:
    display(resumed_embryoscope_data)
else:
    print("No resumed embryoscope timeline data found.")


=== RESUMED EMBRYOSCOPE TIMELINE DATA ===
Records: 1


,prontuario,period_month,embryoscope_usage_count,cumulative_embryoscope_usage,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,embryoscope_missing_payment
0,823589,2024-02,1,1.0,1,2500.0,1.0,2500.0,0.0


In [23]:
# Billing data
print("=== BILLING DATA EMBRYOSCOPE===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Embryoscope'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA EMBRYOSCOPE===
Records: 1


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
33,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2024-02-06,EMBRYOSCOPE,1.0,2500.0,Embryoscope,10101,502,185757,RPS,184500,23,THAIS SANCHES DOMINGUES,874491,,S3563.17,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100017,,,Embryoscope,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,252614,20251118_150534,Mescladas2 (7).xlsx


In [24]:
# Embryoscope treatment data
print("=== EMBRYOSCOPE TREATMENT DATA ===")
embryoscope_treatment_data = results_with_timeline['embryoscope_treatment']
print(f"Records: {len(embryoscope_treatment_data)}")
if not embryoscope_treatment_data.empty:
    display(embryoscope_treatment_data)
else:
    print("No embryoscope treatment data found.")


=== EMBRYOSCOPE TREATMENT DATA ===
Records: 10


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_unit_huntington,embryo_KIDDate,embryo_FertilizationTime,treatment_TreatmentName,patient_unit_huntington_1
0,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
1,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
2,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
3,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
4,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
5,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
6,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
7,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
8,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera
9,PC1P7BHG_45340.4982345602,823589,"MARTARELLI, JULIANA U.",03/09/1985,Ibirapuera,2024-02-24,2024-02-18 13:00:00,2024-233,Ibirapuera


# 4a. Mensalidades

In [25]:
# # Comprehensive embryo timeline data
# print("=== COMPREHENSIVE EMBRYO TIMELINE DATA ===")
# comprehensive_embryo_timeline_data = results_with_timeline['comprehensive_embryo_timeline']
# print(f"Records: {len(comprehensive_embryo_timeline_data)}")
# if not comprehensive_embryo_timeline_data.empty:
#     display(comprehensive_embryo_timeline_data)
# else:
#     print("No comprehensive embryo timeline data found.")


# 4b. Taxa Cryo

In [26]:
# Cryopreservation events timeline data
print("=== CRYOPRESERVATION EVENTS TIMELINE DATA ===")
cryopreservation_events_data = results_with_timeline['cryopreservation_events_timeline']
print(f"Records: {len(cryopreservation_events_data)}")
if not cryopreservation_events_data.empty:
    display(cryopreservation_events_data)
else:
    print("No cryopreservation events timeline data found.")


=== CRYOPRESERVATION EVENTS TIMELINE DATA ===
Records: 2


,prontuario,period_month,freezing_events_count,cumulative_freezing_events,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,events_missing_payment
0,823589,2024-04,0.0,1.0,1.0,2470.0,1.0,2470.0,0.0
1,823589,2024-02,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [27]:
# Cryopreservation events timeline data
print("=== RESUMED CRYOPRESERVATION EVENTS TIMELINE DATA ===")
resumed_cryopreservation_events_data = results_with_timeline['resumed_cryopreservation_events_timeline']
print(f"Records: {len(resumed_cryopreservation_events_data)}")
if not resumed_cryopreservation_events_data.empty:
    display(resumed_cryopreservation_events_data)
else:
    print("No resumed cryopreservation events timeline data found.")

=== RESUMED CRYOPRESERVATION EVENTS TIMELINE DATA ===
Records: 1


,prontuario,period_month,freezing_events_count,cumulative_freezing_events,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,events_missing_payment
0,823589,2024-04,0.0,1.0,1.0,2470.0,1.0,2470.0,0.0


In [28]:
# Billing data
print("=== BILLING DATA TAXA CRYO===")

condicao = (mesclada_data['Descricao'].str.startswith('CRIOPRESERVACAO')) | (mesclada_data['Descrição Gerencial']=='Coleta - Crio')

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA TAXA CRYO===
Records: 1


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
23,823590,RENATO MARTARE,823589,JULIANA UETSUKI MARTARELLI,823589,2024-04-01,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2470.0,Criopreservação,10101,502,193745,RPS,192437,0,CRIOPRESERVACAO,874491,,S3566.04,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,261531,20251118_150534,Mescladas2 (7).xlsx


In [29]:
# Embryo freezing data
print("=== EMBRYO FREEZING DATA ===")
congelamentos_embrioes_data = results_with_timeline['congelamentos_embrioes']
print(f"Records: {len(congelamentos_embrioes_data)}")
if not congelamentos_embrioes_data.empty:
    display(congelamentos_embrioes_data)
else:
    print("No embryo freezing data found.")


=== EMBRYO FREEZING DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,D5,D6,Observacoes,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,24675,E451/24,1,823589,esposa,2024-02-23,1900-01-01 12:00:00,458/24,Não,6,6,E20747,MU,2,None,Vitrificação,1.0,1.0,None,Izadora Reis,Priscila Fabiane Dos Santos Beirigo Paternostro,None,76ed5aa97b623c19ed024b6a6b9bd68e,2025-07-21 21:38:47


In [30]:
# Oocyte freezing data
print("=== OOCYTE FREEZING DATA ===")
congelamentos_ovulos_data = results_with_timeline['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No oocyte freezing data found.")


=== OOCYTE FREEZING DATA ===
Records: 0
No oocyte freezing data found.


In [31]:
# Semen freezing data
print("=== SEMEN FREEZING DATA ===")
congelamentos_semen_data = results_with_timeline['congelamentos_semen']
print(f"Records: {len(congelamentos_semen_data)}")
if not congelamentos_semen_data.empty:
    display(congelamentos_semen_data)
else:
    print("No semen freezing data found.")


=== SEMEN FREEZING DATA ===
Records: 0
No semen freezing data found.


# 5 Consultas

In [32]:
# Consultations timeline data
print("=== CONSULTATIONS TIMELINE DATA ===")
consultas_timeline_data = results_with_timeline['consultas_timeline']
print(f"Records: {len(consultas_timeline_data)}")
if not consultas_timeline_data.empty:
    display(consultas_timeline_data)
else:
    print("No consultations timeline data found.")


=== CONSULTATIONS TIMELINE DATA ===
Records: 1


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,823589.0,2023-11,2,2.0,1,920.0,1.0,1.0,920.0,1.0,1.0


In [33]:
# Resumed consultations timeline data
print("=== RESUMED CONSULTATIONS TIMELINE DATA ===")
resumed_consultas_data = results_with_timeline['resumed_consultas_timeline']
print(f"Records: {len(resumed_consultas_data)}")
if not resumed_consultas_data.empty:
    display(resumed_consultas_data)
else:
    print("No resumed consultations timeline data found.")


=== RESUMED CONSULTATIONS TIMELINE DATA ===
Records: 1


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,823589.0,2023-11,2,2.0,1,920.0,1.0,1.0,920.0,1.0,1.0


In [34]:
# Billing data
print("=== BILLING DATA cCONSULTAS===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Consultas de CRH',
    'Outras Consultas'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA cCONSULTAS===
Records: 1


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
39,823589,JULIANA UETSUK,823589,JULIANA UETSUKI MARTARELLI,823589,2023-11-29,PRIMEIRA CONSULTA REPRODUCAO HUMANA,1.0,920.0,Consultas de CRH,10101,502,176596,RPS,175492,23,THAIS SANCHES DOMINGUES,874489,,S3555.01,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100009,,,1ª Consulta,0,0,1,PRESTACAO DE SERVICO,47,2024-01-15,True,202891,20251118_150534,Mescladas2 (7).xlsx


In [35]:
# Example: Query multiple prontuarios
# prontuarios = ['160751', '171805', '876950']
# all_results = {}
# 
# for pront in prontuarios:
#     print(f"\nQuerying prontuario: {pront}")
#     all_results[pront] = query_patient_data(pront)
# 
# # Compare results
# for pront, results in all_results.items():
#     print(f"Prontuario {pront}: {len(results['tratamentos'])} tratamentos")
